AMPL book Chapter 1

Section 1.6 Adding Resource Constraint

In [3]:
import pandas as pd
from amplpy import AMPL, Environment
#this is for windows only. after dockerizing not required anymore.
#ampl = AMPL(Environment('D:\\amplide.mswin64\\amplide.mswin64')) 
ampl = AMPL(Environment('/opt/ampl.linux64')) 

In [32]:
#ampl.option['solver'] = 'cplexamp'
#we can do this to set the solver we want.
ampl.reset()

In [33]:
ampl.read('./Model4.mod')

In [34]:
ampl.readData('./Model4.dat')

In [35]:
print(ampl.getObjective('Total_Profit').get())
print(ampl.getConstraint('Time').get('reheat'))
print(ampl.getConstraint('Time').get('roll'))

maximize Total_Profit:
	25*X['bands'] + 30*X['coils'] + 29*X['plate'];
subject to Time['reheat']:
	0.005*X['bands'] + 0.005*X['coils'] + 0.005*X['plate'] <= 35;
subject to Time['roll']:
	0.005*X['bands'] + 0.00714286*X['coils'] + 0.00625*X['plate'] <= 40;


In [36]:
ampl.eval('expand Total_Profit;')
ampl.eval('expand Time;')

maximize Total_Profit:
	25*X['bands'] + 30*X['coils'] + 29*X['plate'];

subject to Time['reheat']:
	0.005*X['bands'] + 0.005*X['coils'] + 0.005*X['plate'] <= 35;

subject to Time['roll']:
	0.005*X['bands'] + 0.00714286*X['coils'] + 0.00625*X['plate'] <= 40;



In [37]:
ampl.solve()

MINOS 5.51: optimal solution found.
4 iterations, objective 190071.4286


In [38]:
print(ampl.getObjective('Total_Profit').get().value())

190071.42857142858


##### Note that the profits have increased between TwoVariableLP-Model3.ipynb and ThreeVariableLP-Model3.ipynb by adding the new variable "plate".

In [39]:
ampl.eval('expand X["bands"];')
ampl.eval('expand X["coils"];')
ampl.eval('expand X["plate"];')

Coefficients of X['bands']:
	Time['reheat']   0.005
	Time['roll']     0.005
	Total_Profit    25

Coefficients of X['coils']:
	Time['reheat']   0.005
	Time['roll']     0.00714286
	Total_Profit    30

Coefficients of X['plate']:
	Time['reheat']   0.005
	Time['roll']     0.00625
	Total_Profit    29



In [40]:
ampl.eval('display _nvars, _ncons;')
#displays number of variables and number of constraints

_nvars = 3
_ncons = 2



#### If the system/problem number of constraints is much larger than number of variables; this will result in: "Problem has too few degrees of freedom".  There might be redundancy in these constraint. Use the command expand <Constraint_Name>; and check the constraints generated by AMPL. If "\_ncons" >> "\_nvars" then we may have misunderstood the requirements and unnecessarily defined constraints in the model.


In [41]:
print(ampl.getVariable('X').get('bands'))
print(ampl.getVariable('X').get('coils'))
print(ampl.getVariable('X').get('plate'))
print(ampl.getVariable('X').get('bands').value())
print(ampl.getVariable('X').get('coils').value())
print(ampl.getVariable('X').get('plate').value())

var X['bands'] >=1000, <=6000;
var X['coils'] >=500, <=4000;
var X['plate'] >=750, <=3500;
3357.142857142856
500.0
3142.857142857144


###### Notice how the min_limit > 0 and max_limit > 0. Therefore no production line can be halted. All products must be produced atleast upto the min_limit for each product. In the ThreeVariableLP-Model3b, the min_limit was 0 for all products. We can see how the objective cost function changes due to this and the values of the decision variables have shifted now. 

In [42]:
print(ampl.getData('X'))

   index0    |      X      
  'bands'    | 3357.142857142856
  'coils'    |     500     
  'plate'    | 3142.857142857144



In [26]:
values = ampl.getVariable('X').getValues()
print(values)

   index0    |    X.val    
  'bands'    |     6000    
  'coils'    |     500     
  'plate'    | 1028.5714285714291



In [27]:
df = values.toPandas()
print(df)

             X.val
bands  6000.000000
coils   500.000000
plate  1028.571429


In [28]:
l = values.toList()
print(l)

[('bands', 6000.0), ('coils', 500.0), ('plate', 1028.5714285714291)]


In [29]:
d = values.toDict()
print(d)

{'bands': 6000.0, 'coils': 500.0, 'plate': 1028.5714285714291}


### Shadow Price or Marginal Values or Dual Values

##### the display Time command, shows the Time constraints evaluated after solving. This is called the "marginal values" or "dual values" or "shadow price" associated with the Time constraints. The marginal value of a constraint measure how much the value of the objective would improve if the constraint were relaxed by a small amount. 

In other words, in Model4.dat increase max time limit for Reheat process from $35 to $36 (marginal increase), we will see that the objective cost function value increase by $1800. 

In [30]:
ampl.eval('display Time;')
print(ampl.getConstraint('Time').getValues())

Time [*] :=
  roll  4640
;

   index0    |  Time.dual  
  'reheat'   |      0      
   'roll'    |     4640    



### Quantities associated with decision variable X:

1. ampl.eval('display X.lb;') = lower bound
2. ampl.eval('display X.ub;') = upper bound
3. ampl.eval('display X;') = soultion for variable X
4. ampl.eval('display X.rc;') = reduced costs for X

Scroll below to find more details about reduced costs.

In [31]:
X_result = [
    [
        'bands',
        ampl.getVariable('X').get('bands').lb(), 
        ampl.getVariable('X').get('bands').value(),
        ampl.getVariable('X').get('bands').ub(),
        ampl.getVariable('X').get('bands').rc()
    ],
    [
        'coils',
        ampl.getVariable('X').get('coils').lb(), 
        ampl.getVariable('X').get('coils').value(),
        ampl.getVariable('X').get('coils').ub(),
        ampl.getVariable('X').get('coils').rc()
    ],
    [
        'plate',
        ampl.getVariable('X').get('plate').lb(), 
        ampl.getVariable('X').get('plate').value(),
        ampl.getVariable('X').get('plate').ub(),
        ampl.getVariable('X').get('plate').rc()
    ]
]
df_X_result = pd.DataFrame(X_result)
df_X_result.rename(columns={0:'DecisionVariable_X',
                            1:'X_LowerBound',
                            2:'X_Solution',
                            3:'X_UpperBound',
                            4:'X_ReducedCosts'},
                   inplace=True)
df_X_result.set_index('DecisionVariable_X',inplace=True)
df_X_result

,X_LowerBound,X_Solution,X_UpperBound,X_ReducedCosts
DecisionVariable_X,,,,
bands,1000.0,6000.000000,6000.0,1.800000
coils,500.0,500.000000,4000.0,-3.142857
plate,750.0,1028.571429,3500.0,0.000000


* The lower bound and upper bounds refer to commit and market columns in the .dat file.
* Reduced Cost relates to the bounds of decision variable X, similar to the way how Marginal Values (Shadow Prices) relate limits of a contraint. 
* Therefore we see that:
    * Each increase of a ton on the lower bound (commitment) on coils will reduce profits by approximately $\$$1.86. In other words increase lower bound 500 by a unit to 501, this would decrease objective cost function (profits) by approximately $\$$1.86. 
    * Each one ton decrease on lower bound (commitment) on coils will increase profits by approximately $\$$1.86. In other words decrease lower bound 500 by a unit to 499, this would increase objective cost function (profits) by approximately $\$$1.86.
    * The production levels of bands and plate are between their lower and upper bounds. Thus their reduced costs are approximately $\$$0 and has no effect on the profits (objective cost function). In other words, changing their bounds lower or upper bounds will have no effect on the profits (obj cost function).

| Variables | TwoVariableLP                    | ThreeVariableLP           | ThreeVariableLP-Model3c           | ThreeVariableLP-Model4 |
| :--- |                              |                           |                              | ---: |
| Bands | 6000.0                   | 6000.0            | 6000.0            | 3357.142857142856 |
| Coils | 1399.9999999999993       | 0.0            | 500.0          | 500.0 |
| Plate | -                                | 1599.9999999999993| 1028.571429          | 3142.857142857144 |
| ObjFunc | 192000                         | 196400 | 194828.5714          | 190071.4286 |

In the ThreeVariableLP case, Coils went to 0.0 while Plate increased to 1599.99.
This makes sense because, profit per hour for :
*  bands = 200 tons/hour * 25 dollar/ton = 5000 dollar/hour
*  coils = 140 tons/hour * 30 dollar/ton = 4200 dollar/hour
*  plate = 160 tons/hour * 29 dollar/ton = 4640 dollar/hour

Based on the hourly profits, bands are the most profitable. So we make as much bands as possible but we are limited to make 6000 tons of bands. The next most profitable is plate, whatever remaining hours are available in the week, we make plate till we reach the maximum limit of 3500 tons of plate.

##### In the ThreeVariableLP-Model3c, Coils are produced at a minimum of 500, and Plate and production lowered to 1028.57.

ThreeVariableLP is the case were we consider only cost and try to attain the most profitable outcome. ThreeVariableLP-Model3c is the case where we need to maintain each product's production line because we may not be able to shut down production line for any of the products for a whole week.

#### ThreeVariableLP-Model4

In this case we have two stage production process for all products. That is each product goes through a reheat process and roll process. In the reheat stage, for each product we can process upto 200 tons per hour. However the reheat stage has weekly max time limit of 35 hours. Similarly in the rolling stage, for each product we can process 200 tons of bands per hour, 140 tons of coils per hour and 160 tons of coils per hour. However the rolling stage has a weekly max time limit of 40 hours. Thus in model4 we have 2 constraints one for each stage; which imposes the limited available for each stage.

#### ThreeVariableLP-Model3c vs ThreeVariableLP-Model4

Comparing this session with our previous one, we see that the additional reheat time restriction reduces profits by about $4750, and forces a substantial change in the optimal solution: much higher production of plate and lower production of bands. Moreover, the logic underlying the optimum is no longer so obvious. It is the difficulty of solving LPs by logical reasoning alone that necessitates computer-based systems such as AMPL.